In [ ]:
!git clone https://github.com/vladislavkorenik/image-restoration
!mv image-restoration/* ./

In [112]:
import torch
import numpy as np
from noise import NoisyDataset
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader

modelName = 'multiplicative_bernoulli'
noiseParam = 0.85

# modelName = 'text'
# noiseParam = 0.3

# modelName = 'gaussian'
# noiseParam = 60

def show(img, a):
    plt.figure()
    if a == 'v':
        npimg = torch.squeeze(img).data.cpu().numpy()
    else:
        npimg = torch.squeeze(img).numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))

In [ ]:
from model import NetModel
from train_utils import Train

architecture = NetModel(3, 64)

params = {
    'noise_model': (modelName, noiseParam),
    'crop_size': 64,
    'clean_targs': False,
    'lr': 0.001,
    'epochs': 100,
    'bs': 2,
    'cuda': True
}

trainer = Train(architecture, 'dataset/train', 'dataset/valid', params)
trainer.train()
model = trainer.architecture

torch.save(model,modelName + '.pt')

In [109]:
model = torch.load('trained/' + modelName + '.pt')

In [ ]:
data_ = NoisyDataset('dataset/valid/', crop_size=128, train_noise_model=(modelName, noiseParam), clean_targ=True)
dl = DataLoader(data_, batch_size=1, shuffle=True)

i = 0
for _list in dl:
    plt.figure()
    pred = model(Variable(_list[0].cuda()))
    show(pred, 'v')
    show(_list[0], 't')
    show(_list[-1], 't')
    if i == 0:
        break